In [11]:
%load_ext autoreload
%autoreload 2

import os, sys
# file_dir = os.path.dirname(__file__)
# sys.path.append(file_dir)
sys.path.append('/home/yz685/low_rank_BOPE')
sys.path.append('/home/yz685/low_rank_BOPE/low_rank_BOPE')
import warnings
import math
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import torch
from torch import Tensor
from tqdm.auto import trange

from botorch.models import SingleTaskGP
from botorch.models.transforms import Standardize

from gpytorch import ExactMarginalLogLikelihood
from gpytorch.constraints import GreaterThan
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.priors import GammaPrior
from low_rank_BOPE.bope_class import BopeExperiment
from low_rank_BOPE.test_problems.shapes import AreaUtil, Image
from low_rank_BOPE.src.diagnostics import check_outcome_model_fit, check_util_model_fit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
portfolio_path = "/home/yz685/low_rank_BOPE/low_rank_BOPE/test_problems/portfolio_opt_surrogate/"
state_dict = torch.load(portfolio_path + "portfolio_surrogate_state_dict.pt")

In [5]:
data_list = list()
for i in range(1, 31):
    data_file = os.path.join(
        portfolio_path, "port_evals", "port_n=100_seed=%d" % i)
    data_list.append(torch.load(data_file))

# join the data together
X = torch.cat(
    [data_list[i]["X"] for i in range(len(data_list))], dim=0
).squeeze(-2)
Y = torch.cat(
    [data_list[i]["Y"] for i in range(len(data_list))], dim=0
).squeeze(-2)

In [12]:
# fit GP
noise_prior = GammaPrior(1.1, 0.5)
noise_prior_mode = (noise_prior.concentration - 1) / noise_prior.rate
likelihood = GaussianLikelihood(
    noise_prior=noise_prior,
    batch_shape=[],
    noise_constraint=GreaterThan(
        0.000005,  # minimum observation noise assumed in the GP model
        transform=None,
        initial_value=noise_prior_mode,
    ),
)

model = SingleTaskGP(
    X, Y, likelihood, outcome_transform=Standardize(m=1))
model.load_state_dict(state_dict)

/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/botorch/models/gpytorch.py:119: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/gpytorch/means/constant_mean.py:20: OldVersionWarning: You have loaded a GP model with a ConstantMean  from a previous version of GPyTorch. The mean module parameter `constant` has been renamed to `raw_constant`. Additionally, the shape of `raw_constant` is now *batch_shape, whereas the shape of `constant` was *batch_shape x 1. We have updated the name/shape of the parameter in your state dict, but we recommend that you re-save your model.
  warnings.warn(


<All keys matched successfully>

In [18]:
model.posterior(torch.rand((20,5))).mean

tensor([[ -0.9272],
        [ -0.9981],
        [ -0.2425],
        [ -2.1761],
        [-12.9900],
        [ -2.6111],
        [ -5.5271],
        [ -1.9521],
        [ -1.4344],
        [ -3.7714],
        [ -1.7557],
        [ -0.5026],
        [ -3.0698],
        [ -4.1519],
        [ -2.0303],
        [ -2.1791],
        [ -0.1638],
        [ -2.8808],
        [  1.4820],
        [ -3.7665]], grad_fn=<UnsqueezeBackward0>)